# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Prase-XML" data-toc-modified-id="Prase-XML-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prase XML</a></div><div class="lev1 toc-item"><a href="#Save-Data" data-toc-modified-id="Save-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Save Data</a></div><div class="lev1 toc-item"><a href="#Load-Data" data-toc-modified-id="Load-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Data</a></div><div class="lev1 toc-item"><a href="#Word-Segmentation" data-toc-modified-id="Word-Segmentation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Word Segmentation</a></div><div class="lev1 toc-item"><a href="#Tokenize-Text" data-toc-modified-id="Tokenize-Text-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Tokenize Text</a></div>

# Prase XML

In [67]:
from xml.dom.minidom import parse
import xml.dom.minidom
import json
import os

In [39]:
DOMTree = xml.dom.minidom.parse("/Users/lizhn7/Downloads/EXPERIMENT/COPA/LM/data/copa-all.xml")
items = DOMTree.documentElement.getElementsByTagName('item')

In [52]:
index = []
asksFor = []
label = []
premise = []
alternative1 = []
alternative2 = []
wholeSent1 = []
wholeSent2 = []

In [62]:
for item in items:
    index.append(item.getAttribute('id'))
    asksFor.append(item.getAttribute('asks-for'))
    label.append(item.getAttribute('most-plausible-alternative'))
    premise.append(item.getElementsByTagName('p')[0].childNodes[0].data)
    alternative1.append(item.getElementsByTagName('a1')[0].childNodes[0].data)
    alternative2.append(item.getElementsByTagName('a2')[0].childNodes[0].data)
    if asksFor[-1] == 'cause':
        wholeSent1.append(alternative1[-1] + ' ' + premise[-1])
        wholeSent2.append(alternative2[-1] + ' ' + premise[-1])
    else:
        wholeSent1.append(premise[-1] + ' ' + alternative1[-1])
        wholeSent2.append(premise[-1] + ' ' + alternative2[-1])

In [68]:
tupe = (index, asksFor, label, premise, alternative1, alternative2, wholeSent1, wholeSent2)

# Save Data

In [70]:
def write_to_file(content, name):
    with open(name, 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def gen_data(data):
    for i in range(1000):
        yield {
            'index': data[0][i],
            'asks-for': data[1][i],
            'most-plausible-alternative': data[2][i],
            'premise': data[3][i],
            'alternative1': data[4][i],
            'alternative2': data[5][i],
            'whole-sentence1': data[6][i],
            'whole-sentence2': data[7][i]
        }

for i in gen_data(tupe):
    write_to_file(i, '/Users/lizhn7/Downloads/EXPERIMENT/COPA/LM/data/copa-all.json')

# Load Data

In [240]:
def load_data(path, name):
    """
    Load date from file
    """
    data = []
    input_file = os.path.join(path)
    with open(input_file) as f:
        lines = f.readlines()
    for line in lines:
        item = json.loads(line)
        data.append(item[name])
    return data

def add_token(s):
    """
    Add end token
    """
    s = s.split()
    n = []
    for i in s:
        if i.endswith('.') or i.endswith('!') or i.endswith('?'):
            i += 'endtok'
        n.append(i)
    return ' '.join(n)

In [241]:
wholeSent1 = load_data('/Users/lizhn7/Downloads/EXPERIMENT/COPA/LM/data/copa-all.json', 'whole-sentence1')
wholeSent1 = [add_token(c) for c in wholeSent1]
wholeSent2 = load_data('/Users/lizhn7/Downloads/EXPERIMENT/COPA/LM/data/copa-all.json', 'whole-sentence2')
wholeSent2 = [add_token(c) for c in wholeSent2]

In [242]:
wholeSent1[0]

'The sun was rising.endtok My body cast a shadow over the grass.endtok'

# Word Segmentation

In [243]:
from nltk import regexp_tokenize
from nltk import WordNetLemmatizer
import numpy as np
from tqdm import tqdm

In [244]:
def cut(s):
    """
    Word segmentation
    """
    pattern = r'''
              (?x)                   # set flag to allow verbose regexps 
              (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A. 
              |\w+(?:[-&']\w+)*      # words w/ optional internal hyphens/apostrophe  
            '''  
    return regexp_tokenize(s, pattern=pattern)

def clean(s):
    """
    Clean data
    """
    for i in range(len(s)):
        for d in ['0', '1', '2', '3', '4', '5' ,'6', '7', '8', '9']:
            if d in s[i]:
                s[i] = '0'
        if s[i] == 'p' and i < len(s)-1:
            if s[i+1] == 'm':
                s[i] = 'pm'
                s[i+1] = ''
        if s[i] == 'a' and i < len(s)-1:
            if s[i+1] == 'm':
                s[i] = 'am'
                s[i+1] = ''
        if s[i] == 's':
            s[i] = ''
        if s[i].endswith("'s"):
            s[i] = s[i][:-2]
    return [i for i in s if i != '']

In [245]:
ws1Words = [clean(cut(s)) for s in wholeSent1]
ws2Words = [clean(cut(s)) for s in wholeSent2]

# Tokenize Text